# This cell is auto-generated.

In [383]:
data = 0
while data not in [1, 2]:
    data = int(input('Which data set do you want to use? 1 = Test, 2 = Input '))

if data == 1:
    fp = 'Test/'
    fn = '2224test.txt'
elif data == 2:
    fp = 'Input/'
    fn = '2224input.txt'

f = open(fp + fn, 'r')
raw = [j for j in f.read().splitlines()]

Which data set do you want to use? 1 = Test, 2 = Input 2


In [384]:
raw

['#.########################################################################################################################',
 '#<.>^^^v^^^^<^><>^v^<v<>>v<<<v.vv<<v<v>v.>>>^<<>.<>.v>^v<.v<^^<^<<v.v>><v<>^^^v^.^>.<>vvv>v>>v><v^^>^<^vv>^>v>>^v^>v>^>><#',
 '#>^<^<^<^v^..>^<<>v^>^^<<v^>>vv<>.^.v<^<^..^vv.>>>>v^v>>v^..<>.><v>>^^>vv>.v.>><vv<vv^<v<^^><><.^^.^v.<^^<<..>v<>vv>^>^><#',
 '#..v^^^<^>.>><vv..><vv>^v^>v<v.>^<>>^^<^v>.v>^.>>.<>>v.>.v<>^^<^<vv<>^v<<v^<><.v<<vv>^><<<^^>.<<><v<^^><<><.^<^><>v<vvv.>#',
 '#<^<^v>v^<v<<>vvv^^v^>^.<<><v>><v.^.v.>v..>>^<><<>v.v><^v<<.<><><><<^v.<^>>v^v^vvv><vv<.><.^<vv^^<>>.>>^^v>v^^vv<v<<v><<>#',
 '#<.v>>vv<<vv.v<>^v<^^>>>^<v^>>^vv^vv.>v.>><<v^^<<^<v.^>^<.^>vv^^vv<<>v><^<>.>><^vvv>.>><<vv<vv^vv<^vv>>..^<>>^<vvv><<<^^<#',
 '#>v^>.<<>vvvv<v^<>^<^v<v<<>v^<>^<<v<>v>.v<vv.v<.^>^><<vv<^..^>>^>>^^^<<^v>^<>^>>.>>^v.>^>vv.<.v><^v<^.v.vv^<>^>v.v.><^>>>#',
 '#.>.>^<>^<>><>^.^v<<v<>^>^^<>v<.><^<.v><^v^.v^><>v<>^^^v^vv<.<<vv>v<<<<><<..^^><>^<>v>v.^vv^><.v^vv<v<<>>v^v^

In [385]:
import numpy as np
from copy import deepcopy

In [386]:
entry = (0, raw[0].index('.'))
exit = (len(raw) - 1, raw[len(raw) - 1].index('.'))
minRow = 0
minCol = 0
maxRow = len(raw) - 1
maxCol = len(raw[0]) - 1

In [387]:
def stmover(currLoc, dirTravel):
    global minRow, minCol, maxRow, maxCol
    proposed = tuple(np.add(currLoc, dirTravel))
    if proposed[0] == maxRow:
        proposed = (minRow+1, proposed[1])
    elif proposed[0] == minRow:
        proposed = (maxRow-1, proposed[1])
    elif proposed[1] == maxCol:
        proposed = (proposed[0], minCol+1)
    elif proposed[1] == minCol:
        proposed = (proposed[0], maxCol-1)
    return proposed

In [388]:
storms = {}
stIndex = 0
for r, row in enumerate(raw):
    for c, col in enumerate(np.fromiter(row, (str, 1))):
        if col == '#' or col == '.':
            continue
        else:
            storms[stIndex] = {}
            storms[stIndex]['startLoc'] = (r, c)
            storms[stIndex]['currLoc'] = (r, c)
            if col == '>':
                storms[stIndex]['dirTravel'] = (0, 1)
            elif col == '<':
                storms[stIndex]['dirTravel'] = (0, -1)
            elif col == 'v':
                storms[stIndex]['dirTravel'] = (1, 0)
            elif col == '^':
                storms[stIndex]['dirTravel'] = (-1, 0)
            # below line needs to be able to handle moving off screen
            storms[stIndex]['nextLoc'] = stmover(storms[stIndex]['currLoc'], 
                                                 storms[stIndex]['dirTravel'])
            stIndex += 1

In [389]:
journeys = [entry]

for each item in journeys, evaluate all of the possible moves from that location

In [390]:
def elfMover(currLoc):
    global minRow, minCol, maxRow, maxCol, entry, exit
    possibilities = [(0, 1), (0, -1), (0, 0), (1, 0), (-1, 0)]
    allMoves = []
    for p in possibilities:
        allMoves.append(tuple(np.add(currLoc, p)))
    valMoves = []
    for m in allMoves:
        if m[0]>minRow and m[0]<maxRow and m[1]>minCol and m[1]<maxCol:
            valMoves.append(m)
        if m == entry or m == exit:
            valMoves.append(m)
    return valMoves

In [391]:
def addMoves(journeys, storms):
    global minRow, minCol, maxRow, maxCol
    newJourneys = []
    for j in journeys:
#         print('j = ' + str(j))
        valMoves = elfMover(j)
        for m in valMoves:
#             print('m = ' + str(m))
            if not m in set([storms[x]['nextLoc'] for x in storms]):
                newJourneys.append(m)
    return set(newJourneys)

In [392]:
def advStorms(storms):
    for s in storms:
        storms[s]['currLoc'] = storms[s]['nextLoc']
    for s in storms:        
        storms[s]['nextLoc'] = stmover(storms[s]['currLoc'],
                                       storms[s]['dirTravel'])
    return storms

In [393]:
def runValley(journeys, storms):
    mins = 0
    global exit, entry
    while True:
        journeys = addMoves(journeys, storms)
        storms = advStorms(storms)
        mins += 1
#         print('Mins: ' + str(mins))
        if (exit[0]-1, exit[1]) in set(journeys):
            mins += 1
            storms = advStorms(storms)
            print('Minutes to reach first exit: ' + str(mins))
            journeys = [exit]
            while True:
                journeys = addMoves(journeys, storms)
                storms = advStorms(storms)
                mins += 1
#                 print('Mins: ' + str(mins))
                if (entry[0]+1, entry[1]) in set(journeys):
                    mins += 1
                    storms = advStorms(storms)
                    print('Minutes to return to exit: ' + str(mins))
                    journeys = [entry]
                    while True:
                        journeys = addMoves(journeys, storms)
                        storms = advStorms(storms)
                        mins += 1
#                         print('Mins: ' + str(mins))  
                        if (exit[0]-1, exit[1]) in set(journeys):
                            mins += 1
                            storms = advStorms(storms)
                            print('Minutes to reach second exit: ' + str(mins))
                            return

In [394]:
runValley(journeys, storms)

Minutes to reach first exit: 290
Minutes to return to exit: 562
Minutes to reach second exit: 842
